### Homework 5: Question search engine

Remeber week01 where you used GloVe embeddings to find related questions? That was.. cute, but far from state of the art. It's time to really solve this task using context-aware embeddings.

__Warning:__ this task assumes you have seen `seminar.ipynb`!

In [ ]:
%pip install --upgrade transformers datasets accelerate deepspeed
import torch
import torch.nn as nn
import torch.nn.functional as F
import transformers
import datasets

  error: subprocess-exited-with-error
  
  × Getting requirements to build wheel did not run successfully.
  │ exit code: 1
  ╰─> [23 lines of output]
      [WARNING] Unable to import torch, pre-compiling ops will be disabled. Please visit https://pytorch.org/ to see how to properly install torch on your system.
       [WARNING]  unable to import torch, please install it if you want to pre-compile any deepspeed ops.
      DS_BUILD_OPS=1
      Traceback (most recent call last):
        File "c:\Users\rinok\AppData\Local\Programs\Python\Python312\Lib\site-packages\pip\_vendor\pyproject_hooks\_in_process\_in_process.py", line 353, in <module>
          main()
        File "c:\Users\rinok\AppData\Local\Programs\Python\Python312\Lib\site-packages\pip\_vendor\pyproject_hooks\_in_process\_in_process.py", line 335, in main
          json_out['return_val'] = hook(**hook_input['kwargs'])
                                   ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
        File "c:\Users\rinok\AppData\Local\Pr

     ---------------------------------------- 0.0/43.5 kB ? eta -:--:--
     ----------------- -------------------- 20.5/43.5 kB 640.0 kB/s eta 0:00:01
     -------------------------------------- 43.5/43.5 kB 528.5 kB/s eta 0:00:00
     ---------------------------------------- 0.0/1.4 MB ? eta -:--:--
     -- ------------------------------------- 0.1/1.4 MB 2.6 MB/s eta 0:00:01
     --------- ------------------------------ 0.3/1.4 MB 4.2 MB/s eta 0:00:01
     --------------------- ------------------ 0.8/1.4 MB 6.0 MB/s eta 0:00:01
     ---------------------------------------  1.4/1.4 MB 8.2 MB/s eta 0:00:01
     ---------------------------------------  1.4/1.4 MB 8.2 MB/s eta 0:00:01
     ---------------------------------------- 1.4/1.4 MB 5.3 MB/s eta 0:00:00
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'error'
Note: you ma

### Load data and model

In [ ]:
qqp = datasets.load_dataset('SetFit/qqp')
print('\n')
print("Sample[0]:", qqp['train'][0])
print("Sample[3]:", qqp['train'][3])

Repo card metadata block was not found. Setting CardData to empty.




Sample[0]: {'text1': 'How is the life of a math student? Could you describe your own experiences?', 'text2': 'Which level of prepration is enough for the exam jlpt5?', 'label': 0, 'idx': 0, 'label_text': 'not duplicate'}
Sample[3]: {'text1': 'What can one do after MBBS?', 'text2': 'What do i do after my MBBS ?', 'label': 1, 'idx': 3, 'label_text': 'duplicate'}


In [ ]:
model_name = "gchhablani/bert-base-cased-finetuned-qqp"
tokenizer = transformers.AutoTokenizer.from_pretrained(model_name)
model = transformers.AutoModelForSequenceClassification.from_pretrained(model_name)

### Tokenize the data

In [ ]:
MAX_LENGTH = 128
def preprocess_function(examples):
    result = tokenizer(
        examples['text1'], examples['text2'],
        padding='max_length', max_length=MAX_LENGTH, truncation=True
    )
    result['label'] = examples['label']
    return result

qqp_preprocessed = qqp.map(preprocess_function, batched=True)

Map:   0%|          | 0/40430 [00:00<?, ? examples/s]

In [ ]:
print(repr(qqp_preprocessed['train'][0]['input_ids'])[:100], "...")

[101, 1731, 1110, 1103, 1297, 1104, 170, 12523, 2377, 136, 7426, 1128, 5594, 1240, 1319, 5758, 136,  ...


### Task 1: evaluation (2 points)

We randomly chose a model trained on QQP - but is it any good?

One way to measure this is with validation accuracy - which is what you will implement next.

Here's the interface to help you do that:

In [ ]:
val_set = qqp_preprocessed['validation']
val_loader = torch.utils.data.DataLoader(
    val_set, batch_size=1, shuffle=False, collate_fn=transformers.default_data_collator
)

In [ ]:
for batch in val_loader:
     break  # here be your training code
print("Sample batch:", batch)

with torch.no_grad():
  predicted = model(
      input_ids=batch['input_ids'],
      attention_mask=batch['attention_mask'],
      token_type_ids=batch['token_type_ids']
  )

print('\nPrediction (probs):', torch.softmax(predicted.logits, dim=1).data.numpy())

Sample batch: {'labels': tensor([0]), 'idx': tensor([0]), 'input_ids': tensor([[  101,  2009,  1132,  2170,   118,  4038,  1177,  2712,   136,   102,
          2009,  1132,  1117, 10224,  4724,  1177,  2712,   136,   102,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,   

__Your task__ is to measure the validation accuracy of your model.
Doing so naively may take several hours. Please make sure you use the following optimizations:

- run the model on GPU with no_grad
- using batch size larger than 1
- use optimize data loader with num_workers > 1
- (optional) use [mixed precision](https://pytorch.org/docs/stable/notes/amp_examples.html)


In [ ]:
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
device

'cuda:0'

In [ ]:
from tqdm import tqdm
import numpy as np

model.to(device)

validation_data = qqp_preprocessed['validation']
data_loader = torch.utils.data.DataLoader(
    validation_data, batch_size=32, shuffle=False, collate_fn=transformers.default_data_collator
)

accuracy_per_batch = []
with torch.autocast(device_type=device, dtype=torch.float16):
    for data_batch in tqdm(data_loader):
        with torch.no_grad():
            predictions = model(
                input_ids=data_batch['input_ids'].to(device),
                attention_mask=data_batch['attention_mask'].to(device),
                token_type_ids=data_batch['token_type_ids'].to(device)
            )
            predicted_labels = torch.argmax(torch.softmax(predictions.logits, dim=1), dim=1).cpu().numpy()
            true_labels = data_batch['labels'].numpy()
            accuracy_per_batch.append((predicted_labels == true_labels).mean())

overall_accuracy = np.mean(accuracy_per_batch)
overall_accuracy

  0%|          | 0/1264 [00:00<?, ?it/s]c:\Users\rinok\AppData\Local\Programs\Python\Python312\Lib\site-packages\transformers\models\bert\modeling_bert.py:440: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:555.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(
100%|██████████| 1264/1264 [00:16<00:00, 75.13it/s]


0.9082984855334539

In [ ]:
accuracy = overall_accuracy

In [ ]:
assert 0.9 < accuracy < 0.91

### Task 2: train the model (8 points)

For this task, you have two options:

__Option A:__ fine-tune your own model. You are free to choose any model __except for the original BERT.__ We recommend [DeBERTa-v3](https://huggingface.co/microsoft/deberta-v3-base). Better yet, choose the best model based on public benchmarks (e.g. [GLUE](https://gluebenchmark.com/)).

You can write the training code manually or use transformers.Trainer (see [this example](https://github.com/huggingface/transformers/blob/main/examples/pytorch/text-classification)). Please make sure that your model's accuracy is at least __comparable__ with the above example for BERT.


__Option B:__ compare at least 3 pre-finetuned models (in addition to the above BERT model). For each model, report (1) its accuracy, (2) its speed, measured in samples per second in your hardware setup and (3) its size in megabytes. Please take care to compare models in equal setting, e.g. same CPU / GPU. Compile your results into a table and write a short (~half-page on top of a table) report, summarizing your findings.

## Model №1: ALBERT
Model link: https://huggingface.co/Alireza1044/albert-base-v2-qqp

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, pipeline
from tqdm import tqdm
import torch
import numpy as np

albert_model_name = "Alireza1044/albert-base-v2-qqp"
albert_tokenizer = AutoTokenizer.from_pretrained(albert_model_name)
albert_model = AutoModelForSequenceClassification.from_pretrained(albert_model_name)

albert_model.to(device)

# Используем высокоуровневый pipeline для предсказания
#pipe = pipeline("text-classification", model=albert_model, tokenizer=albert_tokenizer)

# Пора оценить точность
accuracy_batches = []
for data_batch in tqdm(data_loader):
    with torch.no_grad():
        predictions = albert_model(
            input_ids=data_batch['input_ids'].to(device),
            attention_mask=data_batch['attention_mask'].to(device),
            token_type_ids=data_batch['token_type_ids'].to(device)
        )
        predicted = torch.argmax(torch.softmax(predictions.logits, dim=1), dim=1).cpu().numpy()
        actual = data_batch['labels'].numpy()
        accuracy_batches.append((predicted == actual).mean())

albert_accuracy = np.mean(accuracy_batches)
print("ALBERT accuracy:", albert_accuracy)

 21%|██        | 260/1264 [00:10<00:37, 26.83it/s]

model.safetensors:   0%|          | 0.00/46.7M [00:00<?, ?B/s]

100%|██████████| 1264/1264 [00:48<00:00, 26.18it/s]

ALBERT accuracy: 0.6478582730560578


## Model №2: ERNIE
Model link: https://huggingface.co/rajiv003/ernie-finetuned-qqp

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, pipeline

ernie_model_name = "rajiv003/ernie-finetuned-qqp"
ernie_tokenizer = AutoTokenizer.from_pretrained(ernie_model_name)
ernie_model = AutoModelForSequenceClassification.from_pretrained(ernie_model_name)

ernie_model.to(device)

# Используем высокоуровневый pipeline для предсказания
#pipe = pipeline("text-classification", model=ernie_model, tokenizer=ernie_tokenizer)

# Пора оценить точность
for data_batch in tqdm(data_loader):
    with torch.no_grad():
        predictions = ernie_model(
            input_ids=data_batch['input_ids'].to(device),
            attention_mask=data_batch['attention_mask'].to(device),
            token_type_ids=data_batch['token_type_ids'].to(device)
        )
        predicted = torch.argmax(torch.softmax(predictions.logits, dim=1), dim=1).cpu().numpy()
        actual = data_batch['labels'].numpy()
        accuracy_batches.append((predicted == actual).mean())

ernie_accuracy = np.mean(accuracy_batches)
print("ERNIE accuracy:", ernie_accuracy)

100%|██████████| 1264/1264 [00:35<00:00, 36.09it/s]

ERNIE accuracy: 0.5836258900316456


## Model №3: DeBERTa-base
Model link: https://huggingface.co/Tomor0720/deberta-base-finetuned-qqp

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

deberta_model_name = "Tomor0720/deberta-base-finetuned-qqp"
deberta_tokenizer = AutoTokenizer.from_pretrained(deberta_model_name)
deberta_model = AutoModelForSequenceClassification.from_pretrained(deberta_model_name)

qqp_preprocessed = qqp.map(preprocess_function, batched=True)
deberta_model.to(device)

# Создаем загрузчик данных
validation_data = qqp_preprocessed['validation']
data_loader = torch.utils.data.DataLoader(
    validation_data, batch_size=32, shuffle=False, collate_fn=transformers.default_data_collator
)

# Пора оценить точность
accuracy_batches = []
with torch.autocast(device_type=device, dtype=torch.float16):
    for data_batch in tqdm(data_loader):
        with torch.no_grad():
            predictions = deberta_model(
                input_ids=data_batch['input_ids'].to(device),
                attention_mask=data_batch['attention_mask'].to(device),
                token_type_ids=data_batch['token_type_ids'].to(device)
            )
            predicted = torch.argmax(torch.softmax(predictions.logits, dim=1), dim=1).cpu().numpy()
            actual = data_batch['labels'].numpy()
            accuracy_batches.append((predicted == actual).mean())

deberta_accuracy = np.mean(accuracy_batches)
print("DeBERTa-base accuracy:", deberta_accuracy)

Map:   0%|          | 0/363846 [00:00<?, ? examples/s]

model.safetensors:   0%|          | 0.00/557M [00:00<?, ?B/s]

 28%|██▊       | 353/1264 [00:08<00:20, 45.29it/s]c:\Users\rinok\AppData\Local\Programs\Python\Python312\Lib\site-packages\huggingface_hub\file_download.py:139: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\rinok\.cache\huggingface\hub\models--Tomor0720--deberta-base-finetuned-qqp. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
100%|██████████| 1264/1264 

DeBERTa-base accuracy: 0.5334715472423147


# Comparison

| Model name | Accuracy, % | Speed, it/s |
|------------|-------------|------------|
| albert-base-v2-qqp  | __0.65__    | __26.18__  |напи
| ernie-finetuned-qqp | 0.58        | 36.09      |
| deberta-base        | 0.53        | 40.66      |

# Report
During this task, three models were tested: ALBERT (albert-base-v2-qqp), ERNIE (ernie-finetuned-qqp), and DeBERTa-base (deberta-base-finetuned-qqp). Each model was evaluated based on accuracy, processing speed, and size. One of the biggest challenges was finding suitable pretrained models specifically fine-tuned for the QQP task. Many models lacked pretrained weights for this task or sufficient documentation, which limited the options and complicated ensuring equal testing conditions across all models.

The results showed that the ALBERT model performed the best, achieving an accuracy of 65%. This result highlights the architectural advantages of ALBERT, such as parameter efficiency and reduced redundancy, which make it highly effective even with constrained resources. Its processing speed was 26.18 iterations per second, which was slower than ERNIE and DeBERTa but sufficient for practical use. The ERNIE model ranked second in terms of accuracy at 58%, with the fastest processing speed of 36.09 iterations per second. However, its lower accuracy suggests that it may require further fine-tuning or larger datasets to perform better on this task.

DeBERTa had the lowest accuracy of 53% despite its high speed of 40.66 iterations per second. This may be attributed to the model’s reliance on advanced fine-tuning and hyperparameter optimization, which were not extensively explored in this experiment. The relatively low performance across all models could be due to the limited size of the validation dataset, suboptimal adaptation of the models to the QQP task, or computational resource constraints. Overall, ALBERT demonstrated the most balanced performance, making it the best candidate for question pair classification tasks in this setup.

### Task 3: try the full pipeline (4 points)

Finally, it is time to use your model to find duplicate questions.
Please implement a function that takes a question and finds top-5 potential duplicates in the training set. For now, it is fine if your function is slow, as long as it yields correct results.

Showcase how your function works with at least 5 examples.

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from tqdm import tqdm
import torch

best_model_name = "rajiv003/ernie-finetuned-qqp"
tokenizer = AutoTokenizer.from_pretrained(best_model_name)
model = AutoModelForSequenceClassification.from_pretrained(best_model_name)

model.to(device)

# Объединение текстов из тренировочного набора
train_texts = []
train_texts.extend(qqp['train']['text1'])
train_texts.extend(qqp['train']['text2'])

# Функция для поиска дубликатов
def find_duplicates(question, top_k=5):
    """
    Ищет дубликаты для указанного вопроса в тренировочном наборе.

    :param question: Вопрос, для которого ищутся дубликаты.
    :param top_k: Количество возвращаемых дубликатов.
    :return: Список из top_k возможных дубликатов.
    """
    potential_duplicates = []

    with torch.no_grad():
        with torch.autocast(device_type=device, dtype=torch.float16):
            for candidate_text in tqdm(train_texts):
                tokenized_input = tokenizer(
                    candidate_text, question,
                    padding='max_length', max_length=MAX_LENGTH, truncation=True, return_tensors="pt"
                )

                # Предсказываем метку
                predictions = model(
                    input_ids=tokenized_input['input_ids'].to(device),
                    attention_mask=tokenized_input['attention_mask'].to(device)
                )
                predicted_label = torch.argmax(torch.softmax(predictions.logits, dim=1), dim=1).item()

                # Если текст определен как дубликат
                if predicted_label == 1:
                    print("Found duplicate:", candidate_text)
                    potential_duplicates.append(candidate_text)
                    if len(potential_duplicates) >= top_k:
                        break

    return potential_duplicates

# Пример вызова функции с вопросом
example_question = "How one can control impulsive emotions?"
duplicates = find_duplicates(example_question)
print("Top-5 duplicates:", duplicates)

  0%|          | 30/727692 [00:00<1:24:29, 143.53it/s]

Found duplicate: How do I control my horny emotions?


  0%|          | 222/727692 [00:01<1:18:55, 153.63it/s]

Found duplicate: If more vacuum energy appears with expansion and it has no limit, can infinite of this energy be created? If yes is energy infinite?


  0%|          | 670/727692 [00:04<1:19:14, 152.92it/s]

...

Found duplicate: What do I do when I can't control my anger?


  0%|          | 2299/727692 [00:16<1:45:49, 114.24it/s]

Found duplicate: How one can control impulsive emotions?


  1%|          | 3927/727692 [00:28<1:26:32, 139.40it/s]

Found duplicate: What are some ways to control negative thoughts?
Top-5 duplicates: ['How do I control my horny emotions?', 'If more vacuum energy appears with expansion and it has no limit, can infinite of this energy be created? If yes is energy infinite?', "What do I do when I can't control my anger?", 'How one can control impulsive emotions?', 'What are some ways to control negative thoughts?']


__Bonus:__ for bonus points, try to find a way to run the function faster than just passing over all questions in a loop. For isntance, you can form a short-list of potential candidates using a cheaper method, and then run your tranformer on that short list. If you opted for this solution, please keep both the original implementation and the optimized one - and explain briefly what is the difference there.

### Original implementation

In [ ]:
def find_duplicates_original(question, top_k=5):
    """
    Ищет дубликаты, перебирая все вопросы тренировочного набора.
    """
    potential_duplicates = []

    with torch.no_grad():
        with torch.autocast(device_type=device, dtype=torch.float16):
            for candidate_text in tqdm(train_texts):
                # Токенизация текста
                tokenized_input = tokenizer(
                    candidate_text, question,
                    padding='max_length', max_length=MAX_LENGTH, truncation=True, return_tensors="pt"
                )

                # Предсказание метки
                predictions = model(
                    input_ids=tokenized_input['input_ids'].to(device),
                    attention_mask=tokenized_input['attention_mask'].to(device)
                )
                predicted_label = torch.argmax(torch.softmax(predictions.logits, dim=1), dim=1).item()

                # Если текст определен как дубликат
                if predicted_label == 1:
                    potential_duplicates.append(candidate_text)
                    if len(potential_duplicates) >= top_k:
                        break

    return potential_duplicates

### Optimized implementation

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# Предварительная обработка данных: создание TF-IDF векторов
tfidf_vectorizer = TfidfVectorizer(max_features=10000, stop_words="english")
tfidf_matrix = tfidf_vectorizer.fit_transform(train_texts)

def find_duplicates_optimized(question, top_k=5):
    """
    Оптимизированная версия: сначала выбираем кандидатов с помощью TF-IDF, затем применяем трансформер.
    """
    # Шаг 1: Найти ближайшие кандидаты с использованием TF-IDF
    question_vector = tfidf_vectorizer.transform([question])
    similarities = cosine_similarity(question_vector, tfidf_matrix).flatten()

    # Получить индексы top_k самых похожих текстов
    candidate_indices = np.argsort(similarities)[::-1][:100]  # 100 ближайших кандидатов
    shortlisted_candidates = [train_texts[i] for i in candidate_indices]

    # Шаг 2: Применить модель только на shortlist
    potential_duplicates = []

    with torch.no_grad():
        with torch.autocast(device_type=device, dtype=torch.float16):
            for candidate_text in shortlisted_candidates:
                tokenized_input = tokenizer(
                    candidate_text, question,
                    padding='max_length', max_length=MAX_LENGTH, truncation=True, return_tensors="pt"
                )

                predictions = model(
                    input_ids=tokenized_input['input_ids'].to(device),
                    attention_mask=tokenized_input['attention_mask'].to(device)
                )
                predicted_label = torch.argmax(torch.softmax(predictions.logits, dim=1), dim=1).item()

                if predicted_label == 1:
                    potential_duplicates.append(candidate_text)
                    if len(potential_duplicates) >= top_k:
                        break

    return potential_duplicates

In [ ]:
# Original
print("Original implementation:")
duplicates_original = find_duplicates_original("How do I control my horny emotions?")
print("Duplicates:", duplicates_original)

# Optimal
print("\nOptimized implementation:")
duplicates_optimized = find_duplicates_optimized("How do I control my horny emotions?")
print("Duplicates:", duplicates_optimized)

Original implementation:


  1%|          | 4854/727692 [00:33<1:22:41, 145.70it/s]

Duplicates: ['How do I control my horny emotions?', 'If more vacuum energy appears with expansion and it has no limit, can infinite of this energy be created? If yes is energy infinite?', 'How one can control impulsive emotions?', 'How do i control emotions at work place or public place?', 'What is the best way to control our emotions?']

Optimized implementation:
Duplicates: ['How do I control my horny emotions?', 'How do we control our emotions?', "How much control can one have over one's emotions?", 'How do I have control over my emotions?', "How much control can one have over one's emotions?"]
